# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

,0
payment_history,float64
location_parameter,float64
stem_degree_score,float64
gpa_ranking,float64
alumni_success,float64
study_major_code,float64
time_to_completion,float64
finance_workshop_score,float64
cohort_ranking,float64
total_loan_score,float64


In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

,count
credit_ranking,
1,855
0,744


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
#display(y.head())
y.sample(5)


,credit_ranking
1462,1
1566,1
487,1
953,1
189,0


In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [12]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


((1279, 11), (320, 11), (1279,), (320,))

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

X_train_scaled[:5]




array([[-0.73307913,  0.6648928 , -1.25704443, -0.3204585 , -0.45362151,
        -0.74240736, -0.6455073 ,  0.24000129,  0.98846046,  0.0630946 ,
        -0.87223395],
       [ 1.06774091, -0.62346154,  1.52314768,  0.60886277, -0.36954631,
        -1.12518952, -1.11200285,  0.18789883, -1.7535127 , -0.17390392,
        -0.77978452],
       [-1.74604041, -1.07158479, -1.35814232, -0.53491726, -0.78992229,
         1.07580793,  0.53628144, -2.67773653,  2.32756363,  0.77409018,
         3.28799021],
       [-0.62052788,  0.49684658, -1.05484864, -0.0345135 , -0.20139592,
         0.11885252,  1.18937522,  0.37546769,  1.24352773, -0.76640023,
        -0.6873351 ],
       [-0.50797663,  0.60887739, -1.00429969, -0.53491726,  0.26101766,
        -0.74240736, -0.7077067 , -0.33312578, -0.09557544, -0.47015208,
        -0.77978452]])

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model
input_features = X_train_scaled.shape[1]

# Review the number of features
input_features

11

In [18]:
# Define the number of hidden nodes for the first hidden layer

hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer (binary classification, so 1 neuron with sigmoid activation)
output_neurons = 1

(hidden_nodes_layer1, hidden_nodes_layer2, output_neurons)


(64, 32, 1)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer with 64 nodes and 'relu' activation function
model.add(Dense(hidden_nodes_layer1, activation='relu', input_dim=input_features))

# Add the second hidden layer with 32 nodes and 'relu' activation function
model.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer with 1 neuron (for binary classification) and 'sigmoid' activation function
model.add(Dense(output_neurons, activation='sigmoid'))

# Summary of the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# Display the Sequential model summary
# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [21]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [23]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8743 - loss: 0.3187 - val_accuracy: 0.7563 - val_loss: 0.5426
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8582 - loss: 0.3518 - val_accuracy: 0.7844 - val_loss: 0.5426
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8708 - loss: 0.3207 - val_accuracy: 0.7781 - val_loss: 0.5403
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8831 - loss: 0.3106 - val_accuracy: 0.7719 - val_loss: 0.5497
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8736 - loss: 0.3207 - val_accuracy: 0.7781 - val_loss: 0.5448
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8591 - loss: 0.3282 - val_accuracy: 0.7625 - val_loss: 0.5487
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8746 - loss: 0.3126 - val_accuracy: 0.7688 - val_loss: 0.5471
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8644 - loss: 0.3155 - val_accuracy: 0.7719 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [24]:
# Evaluate the model loss and accuracy using the test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)

# Display the results
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7853 - loss: 0.6236 
Test Loss: 0.633104681968689
Test Accuracy: 0.768750011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [25]:
# Set the model's file path
model_file_path = 'student_loans.keras'

# Export the model to a keras file
model.save(model_file_path)

print(f"Model saved to {model_file_path}")

Model saved to student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [26]:
from tensorflow.keras.models import load_model

# Set the model's file path
model_file_path = 'student_loans.keras'

# Load the model to a new object
loaded_model = load_model(model_file_path)

print("Model loaded successfully")

Model loaded successfully


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [30]:
import pandas as pd

# Make predictions with the test data
predictions = (loaded_model.predict(X_test_scaled) > 0.5).astype("int32")  # Thresholding at 0.5 for binary classification

# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted_Credit_Ranking'])

# Display a sample of the predictions
predictions_df.sample(5)



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,Predicted_Credit_Ranking
126,1
169,1
157,1
133,0
239,0


In [31]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Display a sample of the predictions
predictions_df.sample(5)

,Predicted_Credit_Ranking
137,1
187,0
65,0
307,1
189,0


### Step 4: Display a classification report with the y test data and predictions

In [32]:
from sklearn.metrics import classification_report

# Print the classification report comparing y_test with the predictions
print(classification_report(y_test, predictions, target_names=['Class 0', 'Class 1']))


              precision    recall  f1-score   support

     Class 0       0.78      0.73      0.75       154
     Class 1       0.76      0.81      0.78       166

    accuracy                           0.77       320
   macro avg       0.77      0.77      0.77       320
weighted avg       0.77      0.77      0.77       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**

To build a recommendation system for student loans, the following types of data would be necessary:

**Student Demographics**: *Age, education level, location, and family financial background*. These factors often influence loan eligibility and repayment terms.

**Academic Information**: *GPA, field of study, and expected graduation date*. Certain loans are targeted toward specific academic profiles, such as STEM fields or graduate students.

**Financial Situation**: *Family income, student income, scholarships, and existing financial aid. * Understanding a student's current financial position helps determine their ability to repay a loan.

**Loan History**: If the student has taken any loans previously, this data can be used to determine whether they are eligible for more loans and what terms might be appropriate.

**Loan Provider Information**: *Interest rates, repayment terms, grace periods, and eligibility criteria* from different loan providers. This is essential for comparing options and recommending the most appropriate loan for a student’s situation.

**Repayment Success**: Historical data about loan repayment success rates for similar students would help assess which loan options are best suited for a given profile.

This data would help create a personalized recommendation for students based on their financial needs, academic standing, and ability to repay the loan.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

The model I would primarily use would be content-based filtering. Here’s why:

* Content-based filtering relies on the attributes of both the user (student) and the items (loans). In this case, we would use the student’s demographic, academic, and financial data to match them with loans that best fit their profile.
* By analyzing features such as the student's financial background and academic standing, the system can recommend loans based on how well their profile matches loan eligibility criteria, interest rates, and repayment success rates.

Collaborative filtering would be less applicable here, as student loans are highly individualized, and students may not have a history of borrowing that aligns with peers. Context-based filtering could play a secondary role if we include external factors like market trends or government policies, but content-based filtering would be the primary approach.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

1. **Data Privacy and Security**:


* Collecting sensitive data like financial status, loan history, and academic performance poses a significant privacy risk. Students’ personal and financial data must be securely stored and processed in compliance with data protection regulations such as GDPR or FERPA (in the U.S.).

* Ensuring that the system adheres to strict security protocols is critical, as any data breaches could have severe legal and financial consequences for both students and loan providers.

2. **Loan Diversity and Changing Terms**:

* Student loans come in many forms, from federal loans to private loans, with varying interest rates, repayment plans, and eligibility requirements. Additionally, these terms can change based on government policies or market conditions.

* The recommendation system would need to stay up-to-date with dynamic loan terms and policies to ensure that students receive accurate recommendations. Managing and constantly updating this information in real time could be a logistical challenge.

Both challenges are significant because they can impact the reliability of the system and the trust that users place in it.